In [2]:
import pyprind
import pandas as pd
import os
pbar = pyprind.ProgBar(50000)
labels = {'pos':1,'neg':0}
df = pd.DataFrame()
for s in ('test','train'):
    for l in ('pos','neg'):
        path = 'D:/LearnYourself/python/aclImdb_v1/aclImdb/%s/%s' % (s,l)
        for file in os.listdir(path):
            with open(os.path.join(path,file),'r',encoding='UTF-8') as infile:
                txt = infile.read()
            df = df.append([[txt,labels[l]]],ignore_index=True)
            pbar.update()
df.columns = ['review','sentiment']
import numpy as np
np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))
df.to_csv('D:/LearnYourself/python/aclImdb_v1/aclImdb/movie_data.csv',index=False)
df = pd.read_csv('D:/LearnYourself/python/aclImdb_v1/aclImdb/movie_data.csv')
print(df.head(3))


C:\Users\李蓉\AppData\Local\Temp\ipykernel_29860\791884596.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append([[txt,labels[l]]],ignore_index=True)
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:06:54


                                              review  sentiment
0  In 1974, the teenager Martha Moxley (Maggie Gr...          1
1  OK... so... I really like Kris Kristofferson a...          0
2  ***SPOILER*** Do not read this, if you think a...          0


In [1]:

# coding: utf-8
import pandas as pd
df = pd.read_csv('D:/LearnYourself/python/aclImdb_v1/aclImdb/movie_data1.csv')#读出我们放入CSV中的数据

import re
def delete_html(text):
    text = re.sub('<[^>]*>', " ", text)
    emoticion = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',text)
    text = re.sub('[\W]+',' ',text.lower())+''.join(emoticion).replace('-','')
    return text
df['review']= df['review'].apply(delete_html)
def tokenizer(text):
    return text.split()
from nltk.stem.porter import PorterStemmer
def stem_tokenizer(text):
    porter = PorterStemmer()
    return  [porter.stem(word) for word in text.split()]
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')
X_train  = df.loc[:1000,'review'].values
y_train  = df.loc[:1000,'sentiment'].values
X_test  = df.loc[1000:2000,'review'].values
y_test  = df.loc[1000:2000,'sentiment'].values

from  sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(strip_accents=None,lowercase=False,preprocessor=None)
param_grid = [{
        'vect__ngram_range':[(1,1)],
       'vect__stop_words':[stop,None],
        'vect__tokenizer':[tokenizer,stem_tokenizer],
        'vect__use_idf':[False,True],
        'vect__norm':[None,'l1','l2'],
        'clf__penalty':['l1','l2'],
        'clf__C':[1.0,10.0,100.0]
        }]
lr_tfidf = Pipeline([
    ('vect',tfidf),
    ('clf',LogisticRegression(random_state=0))
])
gs_lr_tfidf = GridSearchCV(lr_tfidf,param_grid,scoring='accuracy',cv=5,verbose=1,n_jobs=-1)
gs_lr_tfidf.fit(X_train,y_train)
print('best choice')
print(gs_lr_tfidf.best_params_)
print('Cv score')
print(gs_lr_tfidf.best_score_)

[nltk_data] Error loading stopwords: <urlopen error [Errno 11004]
[nltk_data]     getaddrinfo failed>


UnicodeEncodeError: 'ascii' codec can't encode characters in position 18-19: ordinal not in range(128)

In [10]:
import numpy as np
from nltk.corpus import stopwords

stop = stopwords.words('english')
import re
def delete_html(text):
    text = re.sub('<[^>]*>', " ", text)
    emoticion = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',text)
    text = re.sub('[\W]+',' ',text.lower())+''.join(emoticion).replace('-','')
    token = [w for w in text.split() if w not in stop]
    return token

def stream_doc(path):
    with open(path,'r', encoding='UTF-8') as csv:
        next(csv)
        for line in csv:
            text,lable = line[:-3],int(line[-2])
            yield text,lable

def get_doc(doc_stream,size):
    doc,y = [],[]
    try:
        for _ in range(size):
            text,lable = next(doc_stream)
            doc.append(text)
            y.append(lable)
    except StopIteration:
        return None,None
    return doc,y
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier
vect = HashingVectorizer(decode_error='ignore',n_features=2**21,preprocessor=None,tokenizer=delete_html)
clf = SGDClassifier(loss='log',random_state=1,n_iter_no_change=1)
doc_stream = stream_doc(path='D:/LearnYourself/python/aclImdb_v1/aclImdb/movie_data.csv')

import pyprind
pb = pyprind.ProgBar(45)
classes = np.array([0, 1])
for _ in range(45):
    x_train,y_train = get_doc(doc_stream,size=1000)
    if not x_train:
        break
    x_train = vect.transform(x_train)
    clf.partial_fit(x_train,y_train,classes=np.array([0, 1]))
    pb.update()
x_test,y_test = get_doc(doc_stream,size=5000)
print(x_test)
x_test = vect.transform(x_test)
print(clf.score(x_test,y_test))

InvalidParameterError: The 'loss' parameter of SGDClassifier must be a str among {'squared_hinge', 'perceptron', 'squared_error', 'epsilon_insensitive', 'log_loss', 'modified_huber', 'squared_epsilon_insensitive', 'hinge', 'huber'}. Got 'log' instead.